In [11]:
%matplotlib notebook
import cvxpy as cp
import dccp
import torch
import numpy as np
from cvxpylayers.torch import CvxpyLayer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import svm
from sklearn.metrics import zero_one_loss, confusion_matrix
from scipy.io import arff
import pandas as pd
import time
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.utils import shuffle
import matplotlib.patches as mpatches
import json
import random
import math
import os, psutil
from datetime import datetime

torch.set_default_dtype(torch.float64)
torch.manual_seed(0)
np.random.seed(0)

TRAIN_SLOPE = 1
EVAL_SLOPE = 5
X_LOWER_BOUND = -10
X_UPPER_BOUND = 10
SEED = 0

# Utils

In [12]:
def split_data(X, Y, percentage):
    num_val = int(len(X)*percentage)
    return X[num_val:], Y[num_val:], X[:num_val], Y[:num_val]

def shuffle(X, Y):
    torch.manual_seed(0)
    np.random.seed(0)
    data = torch.cat((Y, X), 1)
    data = data[torch.randperm(data.size()[0])]
    X = data[:, 1:]
    Y = data[:, 0]
    return X, Y

def conf_mat(Y1, Y2):
    num_of_samples = len(Y1)
    mat = confusion_matrix(Y1, Y2, labels=[-1, 1])*100/num_of_samples
    acc = np.trace(mat)
    return mat, acc

def calc_accuracy(Y, Ypred):
    num = len(Y)
    temp = Y - Ypred
    acc = len(temp[temp == 0])*1./num
    return acc

# CCP classes

In [13]:
class CCP:
    def __init__(self, x_dim, batch_size, funcs, scale):
        self.f_derivative = funcs["f_derivative"]
        self.g = funcs["g"]
        self.c = funcs["c"]
        self.batch_size = batch_size
        
        self.x = cp.Variable((batch_size, x_dim))
        self.xt = cp.Parameter((batch_size, x_dim))
        self.r = cp.Parameter((batch_size, x_dim))
        self.w = cp.Parameter(x_dim)
        self.b = cp.Parameter(1)
        self.slope = cp.Parameter(1)

        target = cp.diag(self.x@(self.f_derivative(self.xt, self.w, self.b, self.slope).T))-self.g(self.x, self.w, self.b, self.slope)-self.c(self.x, self.r, x_dim, scale)
        constraints = [self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND]
        self.prob = cp.Problem(cp.Maximize(cp.sum(target)), constraints)
        
    def ccp(self, r):
        """
        numpy to numpy
        """
        self.xt.value = r
        self.r.value = r
        result = self.prob.solve()
        diff = np.linalg.norm(self.xt.value - self.x.value)
        cnt = 0
        while diff > 0.001 and cnt < 100:
            cnt += 1
            self.xt.value = self.x.value
            result = self.prob.solve()
            diff = np.linalg.norm(self.x.value - self.xt.value)/self.batch_size
        return self.x.value
    
    def optimize_X(self, X, w, b, slope):
        """
        tensor to tensor
        """
        w = w.detach().numpy()
        b = b.detach().numpy()
        slope = np.full(1, slope)
        X = X.numpy()
        
        self.w.value = w
        self.b.value = b
        self.slope.value = slope
        return torch.from_numpy(self.ccp(X))

In [14]:
class DELTA():
    
    def __init__(self, x_dim, funcs, scale):
        self.g = funcs["g"]
        self.c = funcs["c"]
        
        self.x = cp.Variable(x_dim)
        self.r = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.w = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.b = cp.Parameter(1, value = np.random.randn(1))
        self.f_der = cp.Parameter(x_dim, value = np.random.randn(x_dim))

        target = self.x@self.f_der-self.g(self.x, self.w, self.b, TRAIN_SLOPE)-self.c(self.x, self.r, x_dim, scale)
        constraints = [self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND]
        objective = cp.Maximize(target)
        problem = cp.Problem(objective, constraints)
        self.layer = CvxpyLayer(problem, parameters=[self.r, self.w, self.b, self.f_der],
                                variables=[self.x])
        
    def optimize_X(self, X, w, b, F_DER):
        return self.layer(X, w, b, F_DER)[0]

# Gain & Cost functions

In [15]:
def score(x, w, b):
    return x@w + b

def f(x, w, b, slope):
    return 0.5*cp.norm(cp.hstack([1, (slope*score(x, w, b) + 1)]), 2)

def g(x, w, b, slope):
    return 0.5*cp.norm(cp.hstack([1, (slope*score(x, w, b) - 1)]), 2)

def c(x, r, x_dim, scale):
    return (scale)*cp.sum_squares(x-r)

def f_derivative(x, w, b, slope):
    return 0.5*cp.multiply(slope*((slope*score(x, w, b) + 1)/cp.sqrt((slope*score(x, w, b) + 1)**2 + 1)), w)
    
def f_batch(x, w, b, slope):
    return 0.5*cp.norm(cp.vstack([np.ones(x.shape[0]), (slope*score(x, w, b) + 1)]), 2, axis=0)

def g_batch(x, w, b, slope):
    return 0.5*cp.norm(cp.vstack([np.ones((1, x.shape[0])), cp.reshape((slope*score(x, w, b) - 1), (1, x.shape[0]))]), 2, axis=0)

def c_batch(x, r, x_dim, scale):
    return (scale)*cp.square(cp.norm(x-r, 2, axis=1))

def f_derivative_batch(x, w, b, slope):
    nablas = 0.5*slope*((slope*score(x, w, b) + 1)/cp.sqrt((slope*score(x, w, b) + 1)**2 + 1))
    return cp.reshape(nablas, (nablas.shape[0], 1))@cp.reshape(w, (1, x.shape[1]))

# Model

In [16]:
class MyStrategicModel(torch.nn.Module):
    def __init__(self, x_dim, batch_size, funcs, funcs_batch, train_slope, eval_slope, scale, strategic=False):
        torch.manual_seed(0)
        np.random.seed(0)

        super(MyStrategicModel, self).__init__()
        self.x_dim = x_dim
        self.batch_size = batch_size
        self.train_slope, self.eval_slope = train_slope, eval_slope
        self.w = torch.nn.parameter.Parameter(math.sqrt(1/x_dim)*(1-2*torch.rand(x_dim, dtype=torch.float64, requires_grad=True)))
        self.b = torch.nn.parameter.Parameter(math.sqrt(1/x_dim)*(1-2*torch.rand(1, dtype=torch.float64, requires_grad=True)))
        self.strategic = strategic
        self.ccp = CCP(x_dim, batch_size, funcs_batch, scale)
        self.delta = DELTA(x_dim, funcs, scale)
        self.ccp_time = 0
        self.total_time = 0

    def forward(self, X, evaluation=False):
        if self.strategic:
            if evaluation:
                t1 = time.time()
                XT = self.ccp.optimize_X(X, self.w, self.b, self.eval_slope)
                self.ccp_time += time.time()-t1
                X_opt = XT
            else:
                t1 = time.time()
                XT = self.ccp.optimize_X(X, self.w, self.b, self.train_slope)
                self.ccp_time += time.time()-t1
                F_DER = self.get_f_ders(XT, self.train_slope)
                X_opt = self.delta.optimize_X(X, self.w, self.b, F_DER) # Xopt should be equal to XT but we do it again for the gradients
            output = self.score(X_opt)
        else:
            output = self.score(X)        
        return output
    
    def optimize_X(self, X, evaluation=False):
        slope = self.eval_slope if evaluation else self.train_slope
        return self.ccp.optimize_X(X, self.w, self.b, slope)
    
    def normalize_weights(self):
        with torch.no_grad():
            norm = torch.sqrt(torch.sum(self.w**2) + self.b**2)
            self.w /= norm
            self.b /= norm

    def score(self, x):
        return x@self.w + self.b
    
    def get_f_ders(self, XT, slope):
        nablas = 0.5*slope*((slope*self.score(XT) + 1)/torch.sqrt((slope*self.score(XT) + 1)**2 + 1))
        return torch.reshape(nablas, (len(nablas), 1))@torch.reshape(self.w, (1, len(self.w)))

    def calc_accuracy(self, Y, Y_pred):
        Y_pred = torch.sign(Y_pred)
        num = len(Y)
        temp = Y - Y_pred
        acc = len(temp[temp == 0])*1./num        
        return acc
    
    def evaluate(self, X, Y):      
        return self.calc_accuracy(Y, self.forward(X, evaluation=True))
    
    def loss(self, Y, Y_pred):
        return torch.mean(torch.clamp(1 - Y_pred * Y, min=0))
    
    def save_model(self, train_errors, val_errors, train_losses, val_losses, info, path, comment=None):
        if comment is not None:
            path += "/" + comment
            
        filename = path + "/model.pt"
        if not os.path.exists(os.path.dirname(filename)):
            os.makedirs(os.path.dirname(filename))
        torch.save(self.state_dict(), filename)
                
        pd.DataFrame(np.array(train_errors)).to_csv(path + '/train_errors.csv')
        pd.DataFrame(np.array(val_errors)).to_csv(path + '/val_errors.csv')
        pd.DataFrame(np.array(train_losses)).to_csv(path + '/train_losses.csv')
        pd.DataFrame(np.array(val_losses)).to_csv(path + '/val_losses.csv')
        
        with open(path + "/info.txt", "w") as f:
            f.write(info)
    
    def load_model(self, filename):
        self.load_state_dict(torch.load(filename))
        self.eval()
    
    def fit(self, path, X, Y, Xval, Yval, opt, opt_kwargs={"lr":1e-3}, batch_size=128, epochs=100, verbose=False, callback=None, comment=None):
        train_dset = TensorDataset(X, Y)
        train_loader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
        test_dset = TensorDataset(Xval, Yval)
        test_loader = DataLoader(test_dset, batch_size=batch_size, shuffle=True)
        
        opt = opt(self.parameters(), **opt_kwargs)

        train_losses = []
        val_losses = []
        train_errors = []
        val_errors = []
        
        best_val_error = 1
        consecutive_no_improvement = 0

        total_time = time.time()
        for epoch in range(epochs):
            t1 = time.time()
            batch = 1
            train_losses.append([])
            train_errors.append([])
            for Xbatch, Ybatch in train_loader:
#                 try:
                opt.zero_grad()
                Ybatch_pred = self.forward(Xbatch)
                l = self.loss(Ybatch, Ybatch_pred)
                l.backward()
                opt.step()
                train_losses[-1].append(l.item())
                with torch.no_grad():
                    e = self.calc_accuracy(Ybatch, Ybatch_pred)
                    train_errors[-1].append(1-e)
                if verbose:
                    print("batch %03d / %03d | loss: %3.5f | err: %3.5f" %
                          (batch, len(train_loader), np.mean(train_losses[-1]), np.mean(train_errors[-1])))
                batch += 1
                if callback is not None:
                    callback()
#                 except:
#                     print("failed")
                
            with torch.no_grad():
                total_loss = 0
                total_error = 0
                batch = 0
                for Xbatch, Ybatch in test_loader:
#                     try:
                    Yval_pred = self.forward(Xbatch, evaluation=True)
                    val_loss = self.loss(Ybatch, Yval_pred).item()
                    total_loss += val_loss
                    val_error = 1-self.calc_accuracy(Ybatch, Yval_pred)
                    total_error += val_error
                    batch += 1
#                     except:
#                         print("failed")
                        
                avg_loss = total_loss/batch
                avg_error = total_error/batch
                val_losses.append(avg_loss)
                val_errors.append(avg_error)
                if avg_error < best_val_error:
                        consecutive_no_improvement = 0
                        best_val_error = avg_error
                        info = "training time in seconds: {}\nepoch: {}\nbatch size: {}\ntrain slope: {}\neval slope: {}\nlearning rate: {}\nvalidation loss: {}\nvalidation error: {}\n".format(
                        time.time()-total_time, epoch, batch_size, self.train_slope, self.eval_slope, opt_kwargs["lr"], avg_loss, avg_error)
                        self.save_model(train_errors, val_errors, train_losses, val_losses, info, path, comment)
                        print("model saved!")

                else:
                    consecutive_no_improvement += 1
                    if consecutive_no_improvement >= 4:
                        break
                    
            t2 = time.time()
            if verbose:
                print("------------- epoch %03d / %03d | time: %03d sec | loss: %3.5f | err: %3.5f" % (epoch + 1, epochs, t2-t1, val_losses[-1], val_errors[-1]))
        
        self.total_time = time.time()-total_time
        print("training time: {} seconds".format(self.total_time)) 
        return train_errors, val_errors, train_losses, val_losses

In [17]:
def gen_sklearn_data(x_dim, N, informative_frac=1, shift_range=1, scale_range=1, noise_frac=0.01):
    torch.manual_seed(0)
    np.random.seed(0)
    n_informative = int(informative_frac*x_dim)
    n_redundant = x_dim - n_informative
    shift_arr = shift_range*np.random.randn(x_dim)
    scale_arr = scale_range*np.random.randn(x_dim)
    X, Y = make_classification(n_samples=N, n_features=x_dim, n_informative=n_informative, n_redundant=n_redundant,
                               flip_y=noise_frac, shift=shift_arr, scale=scale_arr, random_state=0)
    Y[Y == 0] = -1
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    return torch.from_numpy(X), torch.from_numpy(Y)

In [18]:
path = "./models/runtime_varying_batch_size"
epochs = 5
x_dim = 5
scale = 1
X, Y = gen_sklearn_data(x_dim, 1024)
X, Y, Xval, Yval = split_data(X, Y, 0.25)
print(Xval.size())
print("percent of positive samples: {}%".format(100 * len(Y[Y == 1]) / len(Y)))

funcs = {"f": f, "g": g, "f_derivative": f_derivative, "c": c, "score": score}
funcs_batch = {"f": f_batch, "g": g_batch, "f_derivative": f_derivative_batch, "c": c_batch, "score": score}

total = []
ccp = []
for batch_size in (2**np.arange(9)).tolist():
    strategic_model = MyStrategicModel(x_dim, batch_size, funcs, funcs_batch, TRAIN_SLOPE, EVAL_SLOPE, scale=scale, strategic=True)
    strategic_model.fit(path, X, Y, Xval, Yval,
                        opt=torch.optim.Adam, opt_kwargs={"lr": (1e-1)},
                        batch_size=batch_size, epochs=epochs, verbose=True,
                       comment="batched")
    
    total_time = strategic_model.total_time
    ccp_time = strategic_model.ccp_time
    total.append(total_time)
    ccp.append(ccp_time)
    pd.DataFrame(np.array(total)).to_csv(path + '/total_timing_results.csv')
    pd.DataFrame(np.array(ccp)).to_csv(path + '/ccp_timing_results.csv')

torch.Size([256, 5])
percent of positive samples: 50.78125%


C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\l

batch 001 / 768 | loss: 0.31228 | err: 0.00000
batch 002 / 768 | loss: 0.67294 | err: 0.50000
batch 003 / 768 | loss: 0.44863 | err: 0.33333
batch 004 / 768 | loss: 0.81932 | err: 0.50000
batch 005 / 768 | loss: 0.65546 | err: 0.40000
batch 006 / 768 | loss: 0.83054 | err: 0.50000
batch 007 / 768 | loss: 0.71189 | err: 0.42857
batch 008 / 768 | loss: 0.77489 | err: 0.50000
batch 009 / 768 | loss: 0.68879 | err: 0.44444
batch 010 / 768 | loss: 0.61991 | err: 0.40000
batch 011 / 768 | loss: 0.59401 | err: 0.36364
batch 012 / 768 | loss: 0.57693 | err: 0.33333
batch 013 / 768 | loss: 0.53255 | err: 0.30769
batch 014 / 768 | loss: 0.57349 | err: 0.35714
batch 015 / 768 | loss: 0.56817 | err: 0.33333
batch 016 / 768 | loss: 0.53266 | err: 0.31250
batch 017 / 768 | loss: 0.51330 | err: 0.29412
batch 018 / 768 | loss: 0.50176 | err: 0.27778
batch 019 / 768 | loss: 0.56159 | err: 0.31579
batch 020 / 768 | loss: 0.53351 | err: 0.30000
batch 021 / 768 | loss: 0.65893 | err: 0.33333
batch 022 / 7

batch 177 / 768 | loss: 0.53265 | err: 0.23164
batch 178 / 768 | loss: 0.52966 | err: 0.23034
batch 179 / 768 | loss: 0.52670 | err: 0.22905
batch 180 / 768 | loss: 0.53261 | err: 0.23333
batch 181 / 768 | loss: 0.53856 | err: 0.23757
batch 182 / 768 | loss: 0.53560 | err: 0.23626
batch 183 / 768 | loss: 0.53267 | err: 0.23497
batch 184 / 768 | loss: 0.52978 | err: 0.23370
batch 185 / 768 | loss: 0.52691 | err: 0.23243
batch 186 / 768 | loss: 0.52408 | err: 0.23118
batch 187 / 768 | loss: 0.52128 | err: 0.22995
batch 188 / 768 | loss: 0.51851 | err: 0.22872
batch 189 / 768 | loss: 0.51576 | err: 0.22751
batch 190 / 768 | loss: 0.51305 | err: 0.22632
batch 191 / 768 | loss: 0.51036 | err: 0.22513
batch 192 / 768 | loss: 0.50770 | err: 0.22396
batch 193 / 768 | loss: 0.50507 | err: 0.22280
batch 194 / 768 | loss: 0.51253 | err: 0.22680
batch 195 / 768 | loss: 0.51064 | err: 0.22564
batch 196 / 768 | loss: 0.50803 | err: 0.22449
batch 197 / 768 | loss: 0.50545 | err: 0.22335
batch 198 / 7

C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\problems\problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


...

batch 046 / 096 | loss: 0.40894 | err: 0.22826
batch 047 / 096 | loss: 0.40792 | err: 0.22872
batch 048 / 096 | loss: 0.41466 | err: 0.23438
batch 049 / 096 | loss: 0.42247 | err: 0.23724
batch 050 / 096 | loss: 0.42923 | err: 0.24250
batch 051 / 096 | loss: 0.43368 | err: 0.24510
batch 052 / 096 | loss: 0.43023 | err: 0.24279
batch 053 / 096 | loss: 0.43678 | err: 0.24292
batch 054 / 096 | loss: 0.43599 | err: 0.24074
batch 055 / 096 | loss: 0.43499 | err: 0.24091
batch 056 / 096 | loss: 0.43759 | err: 0.24330
batch 057 / 096 | loss: 0.43815 | err: 0.24123
batch 058 / 096 | loss: 0.44126 | err: 0.24353
batch 059 / 096 | loss: 0.44141 | err: 0.24364
batch 060 / 096 | loss: 0.43793 | err: 0.23958
batch 061 / 096 | loss: 0.43620 | err: 0.23770
batch 062 / 096 | loss: 0.43925 | err: 0.23790
batch 063 / 096 | loss: 0.44773 | err: 0.24405
batch 064 / 096 | loss: 0.44663 | err: 0.24023
batch 065 / 096 | loss: 0.45420 | err: 0.24231
batch 066 / 096 | loss: 0.45349 | err: 0.24053
batch 067 / 0

batch 026 / 096 | loss: 0.41568 | err: 0.22596
batch 027 / 096 | loss: 0.41748 | err: 0.22222
batch 028 / 096 | loss: 0.42661 | err: 0.22768
batch 029 / 096 | loss: 0.43356 | err: 0.23276
batch 030 / 096 | loss: 0.42095 | err: 0.22500
batch 031 / 096 | loss: 0.41774 | err: 0.22177
batch 032 / 096 | loss: 0.43351 | err: 0.23047
batch 033 / 096 | loss: 0.43460 | err: 0.23106
batch 034 / 096 | loss: 0.43357 | err: 0.23162
batch 035 / 096 | loss: 0.42761 | err: 0.22857
batch 036 / 096 | loss: 0.43723 | err: 0.23611
batch 037 / 096 | loss: 0.43927 | err: 0.23649
batch 038 / 096 | loss: 0.45926 | err: 0.24671
batch 039 / 096 | loss: 0.46696 | err: 0.25000
batch 040 / 096 | loss: 0.47077 | err: 0.25000
batch 041 / 096 | loss: 0.46954 | err: 0.25000
batch 042 / 096 | loss: 0.46736 | err: 0.25000
batch 043 / 096 | loss: 0.46808 | err: 0.25000
batch 044 / 096 | loss: 0.46138 | err: 0.24716
batch 045 / 096 | loss: 0.45669 | err: 0.24444
batch 046 / 096 | loss: 0.45084 | err: 0.24185
batch 047 / 0

batch 004 / 048 | loss: 0.40645 | err: 0.21875
batch 005 / 048 | loss: 0.44288 | err: 0.25000
batch 006 / 048 | loss: 0.43559 | err: 0.25000
batch 007 / 048 | loss: 0.41899 | err: 0.24107
batch 008 / 048 | loss: 0.40705 | err: 0.23438
batch 009 / 048 | loss: 0.38270 | err: 0.21528
batch 010 / 048 | loss: 0.39610 | err: 0.22500
batch 011 / 048 | loss: 0.39189 | err: 0.22159
batch 012 / 048 | loss: 0.38281 | err: 0.21354
batch 013 / 048 | loss: 0.37714 | err: 0.20673
batch 014 / 048 | loss: 0.37139 | err: 0.20536
batch 015 / 048 | loss: 0.36262 | err: 0.20000
batch 016 / 048 | loss: 0.38403 | err: 0.20703
batch 017 / 048 | loss: 0.39423 | err: 0.21324
batch 018 / 048 | loss: 0.39874 | err: 0.21528
batch 019 / 048 | loss: 0.40433 | err: 0.22039
batch 020 / 048 | loss: 0.40057 | err: 0.21563
batch 021 / 048 | loss: 0.40570 | err: 0.21726
batch 022 / 048 | loss: 0.40793 | err: 0.22159
batch 023 / 048 | loss: 0.40171 | err: 0.21739
batch 024 / 048 | loss: 0.39507 | err: 0.21354
batch 025 / 0

batch 005 / 024 | loss: 0.40092 | err: 0.20000
batch 006 / 024 | loss: 0.36822 | err: 0.18229
batch 007 / 024 | loss: 0.34810 | err: 0.17411
batch 008 / 024 | loss: 0.33274 | err: 0.16797
batch 009 / 024 | loss: 0.33082 | err: 0.16667
batch 010 / 024 | loss: 0.35508 | err: 0.18125
batch 011 / 024 | loss: 0.37408 | err: 0.19034
batch 012 / 024 | loss: 0.38212 | err: 0.19271
batch 013 / 024 | loss: 0.40097 | err: 0.19952
batch 014 / 024 | loss: 0.40751 | err: 0.20536
batch 015 / 024 | loss: 0.40519 | err: 0.20417
batch 016 / 024 | loss: 0.39826 | err: 0.20117
batch 017 / 024 | loss: 0.39600 | err: 0.19853
batch 018 / 024 | loss: 0.39157 | err: 0.19618
batch 019 / 024 | loss: 0.38703 | err: 0.19408
batch 020 / 024 | loss: 0.39412 | err: 0.19687
batch 021 / 024 | loss: 0.39242 | err: 0.19643
batch 022 / 024 | loss: 0.39108 | err: 0.19602
batch 023 / 024 | loss: 0.38772 | err: 0.19429
batch 024 / 024 | loss: 0.39203 | err: 0.19792
model saved!
------------- epoch 002 / 005 | time: 015 sec |

batch 006 / 006 | loss: 0.61406 | err: 0.27214
model saved!
------------- epoch 001 / 005 | time: 009 sec | loss: 0.44625 | err: 0.21484
batch 001 / 006 | loss: 0.45017 | err: 0.22656
batch 002 / 006 | loss: 0.40215 | err: 0.21094
batch 003 / 006 | loss: 0.41924 | err: 0.21615
batch 004 / 006 | loss: 0.42575 | err: 0.21875
batch 005 / 006 | loss: 0.41902 | err: 0.21250
batch 006 / 006 | loss: 0.41467 | err: 0.21094
model saved!
------------- epoch 002 / 005 | time: 010 sec | loss: 0.33653 | err: 0.14844
batch 001 / 006 | loss: 0.33905 | err: 0.17188
batch 002 / 006 | loss: 0.34940 | err: 0.17188
batch 003 / 006 | loss: 0.31919 | err: 0.15365
batch 004 / 006 | loss: 0.35952 | err: 0.16992
batch 005 / 006 | loss: 0.37979 | err: 0.17969
batch 006 / 006 | loss: 0.37987 | err: 0.17969
model saved!
------------- epoch 003 / 005 | time: 010 sec | loss: 0.30664 | err: 0.13281
batch 001 / 006 | loss: 0.47525 | err: 0.21094
batch 002 / 006 | loss: 0.40489 | err: 0.18750
batch 003 / 006 | loss: 0